# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

/opt/conda/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [3]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,3 stars,"The wait time of 30min on a Sunday evening was long considering there were empty tables available. Not enough staff? \nThe servers were very friendly. At the end of the meal, I got a survey on the whole experience including the wait staff. Is that why they were overly friendly? Newly trained employees.\nThe food was ok. I had the Triple B burger and Greek salad. \nI probably won't return anytime soon."
1,1 star,"stay far away from this place, you'll get what you pay for with the 100 dollars for the year membership, they handle the crowds by being as cold and unwelcoming as possible, non-members stay away!!!!!! on Saturdays and Sundays it is extremely crowded and employees will do everything they can to make sure that you do not \""get in the way\"" of the members. (be prepared to get kicked out). It is more like an NRA cult then a gun club, don't waste your time or money, there are plenty of professional gun clubs in AZ do not even consider this one as an option."
2,3 stars,"I go here quite a bit and am always astounded by the poor service given by the owners. I asked for my side salad to be switched out for fries. Their salad is made with cheap lettuce and a disgusting dressing. They can't do it because \""it's a free small salad\"", well then why can't you exchange it for free small fries? The cheap fries from a bag you serve can't cost more than the salad. I'm not asking much, throw 6 fries on my plate and make a customer happy. \n\nThe food is pretty good which keeps me coming back for more and is pretty reasonable. Don't go there expecting quick service though. Considering you order at a counter it shouldn't take as long as it does. \n\nThe falafel is a little greasy and I am not even sure if they are homemade. Their hummus is good though. \n\nI can't account for any of the meat dishes as I am a vegetarian. I also have gotten great service from people who work there who are not the owners. Don't go in expecting to be treated nicely though, go for the cheap mediocre food."
3,3 stars,"Just like any other KFC but they do have a lunch buffet. Right now they're building a gas station next door so theirs only one way in and out and the driveway is kinda narrow. The one downside is that their menu outside is not up to date. So u order one thing and their like \""we don't offer that n e more\"". Well come outside and update ur menu it's not rocket science...smh"
4,5 stars,"I'm a big fan of the pizza here. I LOVE it. I tried a salad and it wasn't for me. The waitress was really cool about offering me something else but I just wanted to stick w/ the pizza. Then the manager came over and offered us a free desert. I thought that was a super class act. I would have given 5 stars for the pizza anyway, but that earned them star # 6 in my book."
5,5 stars,"Fast and friendly service and well prepared, good food needs to be recognized and rewarded regardless of the venue. I am at Denny's in Tempe Arizona. Being the distinguished Elite Yelper that I am, I was going to go out in search of a funky, cool local place to dine but I was so damn tired of traveling that I decided on the Denny's in the parking lot adjacent to my hotel. I was pleasantly surprised!\n\nI went old school diner dinner. A Double Cheeseburger, fries and an old fashioned chocolate shake. YUMMMMMMMMMMMMM!\n\nFresh ingredients, perfectly cooked and fast, friendly service. \n\nThe fact that a \""Tupperware Princess\"" giving a presentation of the latest and greatest food containers did not deter me from sitting at the counter enjoying a great classic diner meal.\n\nGood job Denny's!"
6,4 stars,"I ate here for lunch and a dinner while staying at the New York New York Hotel for a week. For lunch I had a pizza which was almost too much for one person....almost. Fortunately I'd been on planes all day and was starving and the pizza was very good so I finished it all. Service was excellent, personal and attentive. A decent wine selection too. \nFor dinner, i h

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:13<00:00, 3612.14 examples/s]


In [9]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,2 star,"My least favorite of the Caesar Ent hotels. Kind of a small casino. Another comped room so we gave it a try twice. Staff is friendly. Hotel gets very crowded and there is not much walking space. \n\nRooms are decent. Not very big, but they do the trick. Just make sure you do not get rooms with connecting doors. You can hear EVERYTHING going on in the next room (not fun at 6:00 am hearing a drunken argument).\n\nBest thing about the hotel is its center strip. I still wouldn't stay here if I had to pay.","[101, 1422, 1655, 5095, 1104, 1103, 11720, 13832, 1204, 10723, 119, 15994, 1104, 170, 1353, 14330, 119, 2543, 3254, 3537, 1395, 1177, 1195, 1522, 1122, 170, 2222, 3059, 119, 5949, 1110, 4931, 119, 4556, 3370, 1304, 11090, 1105, 1175, 1110, 1136, 1277, 3179, 2000, 119, 165, 183, 165, 183, 2069, 23302, 1116, 1132, 11858, 119, 1753, 1304, 1992, 117, 1133, 1152, 1202, 1103, 7959, 119, 2066, 1294, 1612, 1128, 1202, 1136, 1243, 4045, 1114, 6755, 3581, 119, 1192, 1169, 2100, 142, 17145, 2069, 3663, 24162, 15740, 1280, 1113, 1107, 1103, 1397, 1395, 113, 1136, 4106, 1120, 127, 131, 3135, 1821, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [8]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [10]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [11]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_le

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [12]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="steps", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [18]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
30,1.612000,1.583852,0.310000
60,1.466200,1.251792,0.441000
90,1.115900,1.130412,0.501000
120,1.039300,1.077675,0.522000
150,0.748800,0.967639,0.573000
180,0.709500,0.979449,0.585000


TrainOutput(global_step=189, training_loss=1.0928932422052615, metrics={'train_runtime': 143.6279, 'train_samples_per_second': 20.887, 'train_steps_per_second': 1.316, 'total_flos': 789354427392000.0, 'train_loss': 1.0928932422052615, 'epoch': 3.0})

In [19]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

In [20]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.0569603443145752,
 'eval_accuracy': 0.5,
 'eval_runtime': 0.9862,
 'eval_samples_per_second': 101.397,
 'eval_steps_per_second': 13.182,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [21]:
trainer.save_model(model_dir)

In [22]:
trainer.save_state()

In [23]:
# trainer.model.save_pretrained("./")

## Homework: 使用完整的 YelpReviewFull 数据集训练，看 Acc 最高能到多少

In [19]:
# 设定训练数据为100000，测试数据集为10000
homework_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100000))
homework_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10000))

In [20]:
training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="steps", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=1,
                                  logging_steps=800)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=homework_train_dataset,
    eval_dataset=homework_eval_dataset,
    compute_metrics=compute_metrics,
)

In [22]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
800,1.019100,0.963118,0.572500
1600,0.939700,0.895611,0.597500
2400,0.891000,0.877344,0.615900
3200,0.849300,0.835023,0.634600
4000,0.833200,0.809816,0.638200
4800,0.814600,0.808769,0.643200
5600,0.784600,0.779599,0.656700


TrainOutput(global_step=6250, training_loss=0.866217275390625, metrics={'train_runtime': 3369.7804, 'train_samples_per_second': 29.676, 'train_steps_per_second': 1.855, 'total_flos': 2.63118142464e+16, 'train_loss': 0.866217275390625, 'epoch': 1.0})

In [23]:
trainer.evaluate(homework_eval_dataset)

{'eval_loss': 0.7722072005271912,
 'eval_accuracy': 0.6597,
 'eval_runtime': 96.383,
 'eval_samples_per_second': 103.753,
 'eval_steps_per_second': 12.969,
 'epoch': 1.0}

In [24]:
trainer.save_model(model_dir)